# Import Modules

In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
# %cd '/content/drive/PATH/TO/MNIST_DATA_FILE'

In [3]:
import copy
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score
from mnist.data_utils import load_data

# Utils

In [10]:
def elu(z, alpha=1.0):
    """
    Implement the elu activation function.
    The method takes the input z and returns the output of the function.
    Please DO NOT MODIFY the alpha value.

    Question (a)

    """
    ##### YOUR CODE #####
    return np.where(z > 0, z, alpha * (np.exp(z) - 1))
    #####################


def softmax(X):
    """
    Implement the softmax function.
    The method takes the input X and returns the output of the function.

    Question (a)

    """
    ##### YOUR CODE #####
    e_z = np.exp(X - np.max(X, axis=1, keepdims=True))  # Subtracting np.max(z) for numerical stability
    return e_z / e_z.sum(axis=1, keepdims=True)
    #####################

def deriv_elu(x, alpha=1.0):
    """
    Implement the derivative of elu activation function.
    The method takes the input z and returns the output of the function.
    Please DO NOT MODIFY the alpha value.

    Question (a)

    """
    ##### YOUR CODE #####
    elu_val = np.where(x > 0, x, alpha * (np.exp(x) - 1))
    return np.where(x > 0, 1, elu_val + alpha)
    #####################

def load_batch(X, Y, batch_size, shuffle=True):
    """
    Generates batches with the remainder dropped.

    Do NOT modify this function
    """
    if shuffle:
        permutation = np.random.permutation(X.shape[0])
        X = X[permutation, :]
        Y = Y[permutation, :]
    num_steps = int(X.shape[0])//batch_size
    step = 0
    while step<num_steps:
        X_batch = X[batch_size*step:batch_size*(step+1)]
        Y_batch = Y[batch_size*step:batch_size*(step+1)]
        step+=1
        yield X_batch, Y_batch

# 2-Layer Neural Network

In [11]:
class TwoLayerNN:
    """ a neural network with 2 layers """

    def __init__(self, input_dim, num_hiddens, num_classes):
        """
        Do NOT modify this function.
        """
        self.input_dim = input_dim
        self.num_hiddens = num_hiddens
        self.num_classes = num_classes
        self.params = self.initialize_parameters(input_dim, num_hiddens, num_classes)

    def initialize_parameters(self, input_dim, num_hiddens, num_classes):
        """
        initializes parameters with Xavier Initialization.

        Question (b)
        - refer to https://paperswithcode.com/method/xavier-initialization for Xavier initialization

        Inputs
        - input_dim
        - num_hiddens
        - num_classes
        Returns
        - params: a dictionary with the initialized parameters.
        """
        params = {}
        ##### YOUR CODE #####
        # Xavier initialization for weights
        limit_1 = np.sqrt(6 / float(input_dim + num_hiddens))
        params['W1'] = np.random.uniform(low=-limit_1, high=limit_1, size=(input_dim, num_hiddens))
        params['b1'] = np.zeros((1, num_hiddens))
        
        limit_2 = np.sqrt(6 / float(num_hiddens + num_classes))
        params['W2'] = np.random.uniform(low=-limit_2, high=limit_2, size=(num_hiddens, num_classes))
        params['b2'] = np.zeros((1, num_classes))

        #####################
        return params

    def forward(self, X):
        """
        Define and perform the feed forward step of a two-layer neural network.
        Specifically, the network structure is given by

          y = softmax(ELU(X W1 + b1) W2 + b2)

        where X is the input matrix of shape (N, D), y is the class distribution matrix
        of shape (N, C), N is the number of examples (either the entire dataset or
        a mini-batch), D is the feature dimensionality, and C is the number of classes.

        Question (c)
        - ff_dict will be used to run backpropagation in backward method.

        Inputs
        - X: the input matrix of shape (N, D)

        Returns
        - y: the output of the model
        - ff_dict: a dictionary with all the fully connected units and activations.
        """
        ##### YOUR CODE #####
        W1, b1, W2, b2 = self.params['W1'], self.params['b1'], self.params['W2'], self.params['b2']

        # First fully connected layer
        fc1 = np.dot(X, W1) + b1

        # ELU activation
        elu_out = elu(fc1)

        # Second fully connected layer
        fc2 = np.dot(elu_out, W2) + b2

        # Softmax output
        y = softmax(fc2)

        # Storing the results in a dictionary
        ff_dict = {
            'fc1': fc1,
            'elu_out': elu_out,
            'fc2': fc2,
            'y': y
        }
        #####################
        return y, ff_dict

    def backward(self, X, Y, ff_dict):
        """
        Performs backpropagation over the two-layer neural network, and returns
        a dictionary of gradients of all model parameters.

        Question (d)

        Inputs:
         - X: the input matrix of shape (B, D), where B is the number of examples
              in a mini-batch, D is the feature dimensionality.
         - Y: the matrix of one-hot encoded ground truth classes of shape (B, C),
              where B is the number of examples in a mini-batch, C is the number
              of classes.
         - ff_dict: the dictionary containing all the fully connected units and
              activations.

        Returns:
         - grads: a dictionary containing the gradients of corresponding weights and biases.
        """
        ##### YOUR CODE #####
        W1, W2 = self.params['W1'], self.params['W2']

        # Number of samples
        num_samples = X.shape[0]
    
        # Gradients dictionary
        grads = {}
    
        # Output error
        y_out = ff_dict['y']
        dY = y_out - Y  # Gradient of cross-entropy loss with softmax

        # Backprop through second fully connected layer
        dW2 = np.dot(ff_dict['elu_out'].T, dY) / num_samples
        db2 = np.sum(dY, axis=0, keepdims=True) / num_samples 

        # Backprop through ELU activation
        d_elu = np.dot(dY, W2.T) * deriv_elu(ff_dict['fc1'])

        # Backprop through first fully connected layer
        dW1 = np.dot(X.T, d_elu) / num_samples
        db1 = np.sum(d_elu, axis=0, keepdims=True) / num_samples

        # Store the gradients
        grads['dW1'] = dW1
        grads['db1'] = db1
        grads['dW2'] = dW2
        grads['db2'] = db2
    
        #####################
        return grads


    def compute_loss(self, Y, Y_hat):
        """
        Computes cross entropy loss.

        Do NOT modify this function.

        Inputs
            Y:
            Y_hat:
        Returns
            loss:
        """
        loss = -(1/Y.shape[0]) * np.sum(np.multiply(Y, np.log(Y_hat)))
        return loss

    def train(self, X, Y, X_val, Y_val, lr, n_epochs, batch_size, log_interval=1):
        """
        Runs mini-batch gradient descent.

        Do NOT Modify this method.

        Inputs
        - X
        - Y
        - X_val
        - Y_Val
        - lr
        - n_epochs
        - batch_size
        - log_interval
        """
        for epoch in range(n_epochs):
          for X_batch, Y_batch in load_batch(X, Y, batch_size):
              self.train_step(X_batch, Y_batch, batch_size, lr)
          if epoch % log_interval==0:
              Y_hat, ff_dict = self.forward(X)
              train_loss = self.compute_loss(Y, Y_hat)
              train_acc = self.evaluate(Y, Y_hat)
              Y_hat, ff_dict = self.forward(X_val)
              valid_loss = self.compute_loss(Y_val, Y_hat)
              valid_acc = self.evaluate(Y_val, Y_hat)
              print('epoch {:02} - train loss/acc: {:.3f} {:.3f}, valid loss/acc: {:.3f} {:.3f}'.\
                    format(epoch, train_loss, train_acc, valid_loss, valid_acc))

    def train_step(self, X_batch, Y_batch, batch_size, lr):
        """
        Updates the parameters using gradient descent.

        Do NOT Modify this method.

        Inputs
        - X_batch
        - Y_batch
        - batch_size
        - lr
        """
        _, ff_dict = self.forward(X_batch)
        grads = self.backward(X_batch, Y_batch, ff_dict)
        self.params["W1"] -= lr * grads["dW1"]/batch_size
        self.params["b1"] -= lr * grads["db1"]/batch_size
        self.params["W2"] -= lr * grads["dW2"]/batch_size
        self.params["b2"] -= lr * grads["db2"]/batch_size

    def evaluate(self, Y, Y_hat):
        """
        Computes classification accuracy.

        Do NOT modify this function

        Inputs
        - Y: A numpy array of shape (N, C) containing the softmax outputs,
             where C is the number of classes.
        - Y_hat: A numpy array of shape (N, C) containing the one-hot encoded labels,
             where C is the number of classes.

        Returns
            accuracy: the classification accuracy in float
        """
        classes_pred = np.argmax(Y_hat, axis=1)
        classes_gt = np.argmax(Y, axis=1)
        accuracy = float(np.sum(classes_pred==classes_gt)) / Y.shape[0]
        return accuracy

# Load MNIST

In [6]:
X_train, Y_train, X_test, Y_test = load_data()

idxs = np.arange(len(X_train))
np.random.shuffle(idxs)
split_idx = int(np.ceil(len(idxs)*0.8))
X_valid, Y_valid = X_train[idxs[split_idx:]], Y_train[idxs[split_idx:]]
X_train, Y_train = X_train[idxs[:split_idx]], Y_train[idxs[:split_idx]]
print()
print('Set validation data aside')
print('Training data shape: ', X_train.shape)
print('Training labels shape: ', Y_train.shape)
print('Validation data shape: ', X_valid.shape)
print('Validation labels shape: ', Y_valid.shape)

MNIST data loaded:
Training data shape: (60000, 784)
Training labels shape: (60000, 10)
Test data shape: (10000, 784)
Test labels shape: (10000, 10)

Set validation data aside
Training data shape:  (48000, 784)
Training labels shape:  (48000, 10)
Validation data shape:  (12000, 784)
Validation labels shape:  (12000, 10)


# Training & Evaluation

In [7]:
###
# Question (e)
# Tune the hyperparameters with validation data,
# and print the results by running the lines below.
###

In [12]:
# model instantiation
model = TwoLayerNN(input_dim=784, num_hiddens=128, num_classes=10)

In [27]:
# train the model
lr, n_epochs, batch_size = 0.5, 50, 32
model.train(X_train, Y_train, X_valid, Y_valid, lr, n_epochs, batch_size)

epoch 00 - train loss/acc: 0.116 0.968, valid loss/acc: 0.152 0.956
epoch 01 - train loss/acc: 0.113 0.969, valid loss/acc: 0.151 0.956
epoch 02 - train loss/acc: 0.107 0.971, valid loss/acc: 0.145 0.958
epoch 03 - train loss/acc: 0.102 0.972, valid loss/acc: 0.140 0.958
epoch 04 - train loss/acc: 0.098 0.973, valid loss/acc: 0.138 0.960
epoch 05 - train loss/acc: 0.095 0.974, valid loss/acc: 0.135 0.960
epoch 06 - train loss/acc: 0.091 0.975, valid loss/acc: 0.131 0.961
epoch 07 - train loss/acc: 0.088 0.976, valid loss/acc: 0.129 0.962
epoch 08 - train loss/acc: 0.086 0.976, valid loss/acc: 0.127 0.963
epoch 09 - train loss/acc: 0.084 0.977, valid loss/acc: 0.125 0.963
epoch 10 - train loss/acc: 0.081 0.978, valid loss/acc: 0.123 0.963
epoch 11 - train loss/acc: 0.078 0.979, valid loss/acc: 0.121 0.964
epoch 12 - train loss/acc: 0.076 0.979, valid loss/acc: 0.119 0.966
epoch 13 - train loss/acc: 0.074 0.980, valid loss/acc: 0.117 0.967
epoch 14 - train loss/acc: 0.071 0.981, valid lo

In [28]:
# evalute the model on test data
Y_hat, _ = model.forward(X_test)
test_loss = model.compute_loss(Y_test, Y_hat)
test_acc = model.evaluate(Y_test, Y_hat)
print("Final test loss = {:.3f}, acc = {:.3f}".format(test_loss, test_acc))

Final test loss = 0.081, acc = 0.975



### Question (e) attempts:

lr, n_epochs, batch_size = 0.1, 50, 256
Final test loss = 0.383, acc = 0.895

lr, n_epochs, batch_size = 0.3, 50, 256
Final test loss = 0.417, acc = 0.888

lr, n_epochs, batch_size = 0.1, 50, 128
Final test loss = 0.324, acc = 0.909

lr, n_epochs, batch_size = 0.3, 50, 128
Final test loss = 0.280, acc = 0.921

lr, n_epochs, batch_size = 0.5, 50, 128
Final test loss = 0.239, acc = 0.933

lr, n_epochs, batch_size = 0.5, 50, 64
Final test loss = 0.134, acc = 0.962.

lr, n_epochs, batch_size = 0.5, 50, 32
Final test loss = 0.081, acc = 0.975     # Best result

lr, n_epochs, batch_size = 0.01, 100, 256
Final test loss = 0.378, acc = 0.896